In [1]:
!pip install sentence_transformers openai unstructured
!pip install plotly
!pip install langchain
!pip install tiktoken
!pip install matplotlib
%pip install -Uqqq rich openai tiktoken wandb langchain unstructured tabulate pdf2image chromadb gradio faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.6 MB/s eta 0:00:00
   ━

In [2]:
!pip install linkify-it-py

In [3]:
import numpy as np
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown

In [4]:
import torch
import sys
import csv

csv.field_size_limit(sys.maxsize)

# torch.has_mps

131072

In [6]:
# get a groq api
if os.getenv("GROQ_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["GROQ_API_KEY"] = getpass("")

assert os.getenv("GROQ_API_KEY", "").startswith("gsk"), "This doesn't look like a valid groq API key"
print("groq API key configured")

··········
groq API key configured


#Chunking and RAG

In [11]:
#get a hugging face api key
if os.getenv("Huggingface_api_key") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["Huggingface_api_key"] = getpass("")

assert os.getenv("Huggingface_api_key", "").startswith("hf_"), "This doesn't look like a valid huggingface API key"
print("huggingface API key configured")

··········
huggingface API key configured


In [17]:
from langchain_huggingface import HuggingFaceEmbeddings
import nltk
nltk.download('punkt')
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
txt = '''I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought'''

In [19]:
from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [20]:
encoded_text = tokenizer.encode(txt, padding=True, truncation=True, return_tensors='pt')

In [21]:
with torch.no_grad():
    model_output = model(encoded_text)
    encoded_representation = model_output.last_hidden_state.mean(dim=1)

In [22]:
encoded_text.shape

torch.Size([1, 512])

In [23]:
decoded_text = tokenizer.decode(encoded_text[0], skip_special_tokens=True)

In [24]:
decoded_text

'i would like to get your all thoughts on the bond yield increase this week. i am not worried about the market downturn but the sudden increase in yields. on 2 / 16 the 10 year bonds yields increased by almost 9 percent and on 2 / 19 the yield increased by almost 5 percent. key points from the cnbc article : * * * the “ taper tantrum ” in 2013 was a sudden spike in treasury yields due to market panic after the federal reserve announced that it would begin tapering its quantitative easing program. * * * * * major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. * * * * * however, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation. * * the recent rise in bond yields and u. s. inflation expectations has some investors wary that a re

In [25]:
with open('article.txt', 'w') as f:
  f.write(txt)

In [26]:
MODEL_NAME = "text-davinci-003"

In [ ]:
# from langchain.document_loaders import DirectoryLoader

# def find_md_files(directory):
#     "Find all markdown files in a directory and return a LangChain Document"
#     dl = DirectoryLoader(directory, "**/*.txt")
#     return dl.load()

# documents = find_md_files('data')
# len(documents)

In [ ]:
# documents

#upload files

In [27]:
import pandas as pd

In [28]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

In [29]:
df= pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/df_embed.csv')

In [30]:
!mkdir data

In [32]:
df[:10].to_csv('data/df_embed.csv',index=False)

In [33]:
df.content[0]

'divpelijah is one of my favorite contemporary philosophers here is a relnoopener hrefhttpsconversationswithtylercomepisodeselijahmillgram target_blankthe audio video and transcripta\xa0 here is part of the summaryp\nblockquotepelijah joined tyler to discuss newcombs paradox the reason he doesnt have an opinion about everything the philosophy of dave barry style and simulation theory why philosophers arent often consulted about current events his best stories from taing for robert nozick the sociological correlates of knowing formal logic the question of whether people are more interested in truth or being interesting philosophical cycles what makes nietzsche important today the role that meaning can play in a persons personality and life mill on bentham the idea of true philosophy as dialogue the extent to which modern philosophers are truly philosophical why he views aesthetics as critical to philosophy and morepblockquote\nphere is one excerptp\nblockquote\npstrongcowenstrong\xa0a r

In [34]:
path= 'data/df_embed.csv'
loader = CSVLoader(file_path=path,source_column="title")

data = loader.load()

In [35]:
print(data[0])

page_content='_id: 60d2798bef286f5055d44785
title: My Conversation with Elijah Millgram
content: divpelijah is one of my favorite contemporary philosophers here is a relnoopener hrefhttpsconversationswithtylercomepisodeselijahmillgram target_blankthe audio video and transcripta  here is part of the summaryp
blockquotepelijah joined tyler to discuss newcombs paradox the reason he doesnt have an opinion about everything the philosophy of dave barry style and simulation theory why philosophers arent often consulted about current events his best stories from taing for robert nozick the sociological correlates of knowing formal logic the question of whether people are more interested in truth or being interesting philosophical cycles what makes nietzsche important today the role that meaning can play in a persons personality and life mill on bentham the idea of true philosophy as dialogue the extent to which modern philosophers are truly philosophical why he views aesthetics as critical to 

In [36]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 10 document(s) in your data
There are 5109 characters in your document


In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [38]:
len(texts)

73

In [39]:
texts[9]

Document(metadata={'source': 'My Conversation with the very very smart David Deutsch', 'row': 1}, page_content='pso an everett universe is really a misnomer because what we see in real life is an everett bubble within the universe everything outside the bubble is as it was its undifferentiated or to be exact its exactly as differentiated as it was before then as the bubble spreads out the universe becomes or the multiverse becomes more differentiated but the bubble is always finitep\npstrongcowen\xa0stronghow do your views relate to the philosophical modal realism of\xa0a relnoopener hrefhttpsplatostanfordeduentriesdavidlewis target_blank datahrefhttpsplatostanfordeduentriesdavidlewisdavid lewisap')

In [45]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
db = FAISS.from_documents(texts, embeddings)

In [46]:
query = ' How do you think the many-worlds interpretation of quantum mechanics relates to the view that, just in terms of space, the size of our current universe is infinite, and therefore everything possible is happening in it?'

In [47]:
def get_response_from_query(db, query, k=5):
    """
    text-davinci-003 can handle up to 4097 tokens. Setting the chunksize to 1000 and k to 4 maximizes the number of tokens to analyze.
    """

    docs = db.similarity_search(query, k=k)

    docs_page_content = " ".join([d.page_content for d in docs])

    llm = ChatGroq(model_name = 'Gemma2-9b-It',temperature=0)

    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        A bot that is open to discussions about different cultural, philosophical and political exchanges. I will use do different analysis to the articles provided to me. Stay truthful and if you weren't provided any resources give your oppinion only.
        Answer the following question: {question}
        By searching the following articles: {docs}

        Only use the factual information from the documents. Make sure to mention key phrases from the articles.

        If you feel like you don't have enough information to answer the question, say "I don't know".

        """,
    )

    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run(question=query, docs=docs_page_content,return_source_documents=True)
    r_text = str(response)

    ##evaluation part

    prompt_eval = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        You job is to evaluate if the response to a question is similar to the source given.

        for the following: {question}
        By searching the following article: {docs}

       Give a reason why they are similar or not, start with a Yes or a No.

        """,
    )

    chain_part_2 = LLMChain(llm=llm, prompt=prompt_eval)

    evals = chain_part_2.run(question=r_text, docs=docs_page_content)


    return response,docs,evals

In [48]:
answer,sources,evals=get_response_from_query(db,query,5)

<ipython-input-47-6c818cc985c7>:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-47-6c818cc985c7>:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(question=query, docs=docs_page_content,return_source_documents=True)


In [49]:
print("\n\n> Question:")
print(query)
print("\n> Answer:")
print(answer)
print("\n> Eval:")
print(evals)

# # Print the relevant sources used for the answer
print("----------------------------------SOURCE DOCUMENTS---------------------------")
for document in sources:
    print("\n> " + document.metadata["source"])
    print(document.page_content[:1000])
print("----------------------------------SOURCE DOCUMENTS---------------------------")





> Question:
 How do you think the many-worlds interpretation of quantum mechanics relates to the view that, just in terms of space, the size of our current universe is infinite, and therefore everything possible is happening in it?

> Answer:
According to David Deutsch, the "Everett bubble" within the universe is finite.  He states, "what we see in real life is an Everett bubble within the universe. Everything outside the bubble is as it was its undifferentiated or to be exact its exactly as differentiated as it was before then as the bubble spreads out the universe becomes or the multiverse becomes more differentiated but the bubble is always finite." 

Deutsch also clarifies that the "influence" of an event, like a quantum measurement, cannot spread faster than the speed of light. This suggests that while there may be an infinite number of possibilities, they are not all happening simultaneously in our observable universe. 


He draws a distinction between the "Everettian notion of

In [52]:
import gradio as gr

In [54]:
def greet(query):

    answer,sources,evals = get_response_from_query(db,query,2)
    return answer,sources,evals
examples = [
    ["How to be happy"],
    ["Climate Change Challenges in Europe"],
    ["Philosophy in the world of Minimalism"],
    ["Hate Speech  vs Freedom of Speech"],
    ["Articles by Noam Chomsky on US Politics"],
    ["The importance of values and reflection"]
    ]
demo = gr.Interface(fn=greet, title="cicero-semantic-search", inputs="text",
                    outputs=[gr.components.Textbox(lines=3, label="Response"),
                             gr.components.Textbox(lines=3, label="Source"),
                             gr.components.Textbox(lines=3, label="Evaluation")],
                   examples=examples)

demo.launch(share=True)




Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a26e43a36ab0747cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
